In [ ]:
import json
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel


class DocumentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

        # Создание словаря для преобразования меток в числовые индексы
        self.label_to_index = {label: index for index, label in enumerate(set(labels))}

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = str(self.texts[index])
        label = self.labels[index]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )

        # Преобразование метки в числовой индекс
        label_index = self.label_to_index[label]

        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label_index, dtype=torch.long)
        }

In [ ]:
class DocumentClassifier(nn.Module):
    def __init__(self, num_classes):
        super(DocumentClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-multilingual-cased')
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        _, pooled_output = self.bert(input_ids=input_ids, attention_mask=attention_mask, return_dict=False)
        dropout_output = self.dropout(pooled_output)
        logits = self.fc(dropout_output)
        return logits

In [ ]:
def train(model, dataloader, optimizer, criterion, device):
    model.train()
    train_loss = 0
    train_acc = 0
    for batch in dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        optimizer.zero_grad()
        logits = model(input_ids, attention_mask)
        loss = criterion(logits, labels)
        train_loss += loss.item()

        _, preds = torch.max(logits, 1)
        train_acc += (preds == labels).sum().item()

        loss.backward()
        optimizer.step()

    return train_loss / len(dataloader), train_acc / len(dataloader.dataset)

In [ ]:
def evaluate(model, dataloader, criterion, device):
    model.eval()
    val_loss = 0
    val_acc = 0
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            logits = model(input_ids, attention_mask)
            loss = criterion(logits, labels)
            val_loss += loss.item()

            _, preds = torch.max(logits, 1)
            val_acc += (preds == labels).sum().item()

    return val_loss / len(dataloader), val_acc / len(dataloader.dataset)

Dataset

In [ ]:
with open('normalized_output.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# Извлечение текстов документов и их классов
texts = [item['normalized_content'] for item in data]
labels = [item['type'] for item in data]
#print (labels)
# Разделение данных на обучающую и тестовую выборки
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.4, random_state=42)
# Инициализация токенизатора и модели BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
max_length = 512
# Создание датасетов и даталоадеров
train_dataset = DocumentDataset(train_texts, train_labels, tokenizer, max_length)
test_dataset = DocumentDataset(test_texts, test_labels, tokenizer, max_length)
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=False)

Train

In [ ]:
# Инициализация модели и обучение
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_classes = len(set(labels))
model = DocumentClassifier(num_classes).to(device)
optimizer = optim.AdamW(model.parameters(), lr=2e-5)
criterion = nn.CrossEntropyLoss()
num_epochs = 7

for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    train_loss, train_acc = train(model, train_dataloader, optimizer, criterion, device)
    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
    val_loss, val_acc = evaluate(model, test_dataloader, criterion, device)
    print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
    print()

# Оценка модели на тестовых данных
model.eval()
test_preds = []
with torch.no_grad():
    for batch in test_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        logits = model(input_ids, attention_mask)
        _, preds = torch.max(logits, 1)
        test_preds.extend(preds.cpu().numpy())

# Преобразование числовых индексов меток обратно в строковые метки
index_to_label = {index: label for label, index in train_dataset.label_to_index.items()}
test_preds_labels = [index_to_label[index] for index in test_preds]

# Сохранение обученной модели
torch.save(model.state_dict(), 'trained_model_final.pth')
print("Обученная модель сохранена в файл 'trained_model_final.pth'")

test_acc = accuracy_score(test_labels, test_preds_labels)
print(f"Test Accuracy: {test_acc:.4f}")
print(classification_report(test_labels, test_preds_labels))

In [ ]:
# Классификация документов и добавление поля guessclass
model.eval()
with torch.no_grad():
    for item in data:
        text = item['normalized_content']
        encoding = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=max_length,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )
        input_ids = encoding['input_ids'].to(device)
        attention_mask = encoding['attention_mask'].to(device)
        logits = model(input_ids, attention_mask)
        _, pred = torch.max(logits, 1)
        guessclass = pred.item()

        # Добавление guessclass после ключа type
        index = list(item.keys()).index('type') + 1
        item_keys = list(item.keys())
        item_values = list(item.values())
        item_keys.insert(index, 'guessclass')
        item_values.insert(index, guessclass)
        updated_item = dict(zip(item_keys, item_values))
        item.clear()
        item.update(updated_item)

        # Вывод значений каждого уникального ключа
        unique_keys = set(item.keys())
        for key in unique_keys:
            value = item.get(key)
            print(f"{key}: {value}")

**Нормализация**

In [ ]:
import nltk

nltk.download('stopwords')

In [ ]:
import json
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
from razdel import tokenize
from multiprocessing import Pool
from functools import partial

# Загрузка стоп-слов
stop_words = set(stopwords.words('russian'))

# Добавление стоп-слов, характерных для юридических документов
legal_stop_words = {
    'дом', 'корпус', 'строение', 'квартира', 'офис', 'банк',
    'счет', 'бик', 'инн', 'кпп', 'номер', 'дата', 'год', 'месяц', 'день',
    'рубль', 'копейка', 'тысяча', 'миллион', 'процент', 'штука', 'кг', 'ндс', 'i', 'ii', 'iii', 'n', 'г'
}

# Инициализация морфологического анализатора
morph = MorphAnalyzer()


def normalize_text(text):
    # Токенизация текста
    tokens = [token.text.lower() for token in tokenize(text)]

    # Удаление стоп-слов, знаков препинания
    filtered_tokens = [token for token in tokens if token.isalpha() and token not in stop_words]

    # Лемматизация токенов
    lemmatized_tokens = [morph.parse(token)[0].normal_form for token in filtered_tokens]

    # Удаление юридических стоп-слов
    filtered_lemmatized_tokens = [token for token in lemmatized_tokens if token not in legal_stop_words]

    # Объединение токенов обратно в текст
    normalized_text = ' '.join(filtered_lemmatized_tokens)

    return normalized_text


def process_item(item):
    if 'content' in item:
        item['normalized_content'] = normalize_text(item['content'])
    return item


if __name__ == '__main__':
    # Чтение данных из файла output.json
    with open('check_all.json', 'r', encoding='utf-8') as file:
        data = json.load(file)

    # Многопоточная обработка данных
    with Pool() as pool:
        processed_data = pool.map(process_item, data)

    # Сохранение обработанных данных в новый файл
    with open('normalized_check.json', 'w', encoding='utf-8') as file:
        json.dump(processed_data, file, ensure_ascii=False, indent=4)

**Инференс**

In [ ]:
import json
import torch
from transformers import BertTokenizer
from train_model import DocumentClassifier, max_length


def analyze_text(text, model_path, label_to_index):
    # Загрузка обученной модели
    num_classes = len(label_to_index)
    model = DocumentClassifier(num_classes)
    model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
    model.eval()

    # Инициализация токенизатора
    tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

    # Перенос модели на устройство (CPU или GPU)
    device = torch.device('cpu')
    model.to(device)

    # Создание обратного словаря для сопоставления индексов с метками классов
    index_to_label = {index: label for label, index in label_to_index.items()}
    text = normalize_text(text)

    # Токенизация и подготовка входных данных
    encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=max_length,
        return_token_type_ids=False,
        padding='max_length',
        return_attention_mask=True,
        return_tensors='pt',
        truncation=True
    )
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    # Получение предсказаний модели
    with torch.no_grad():
        outputs = model(input_ids, attention_mask)
        _, predicted_class_index = torch.max(outputs, 1)

    # Преобразование индекса предсказанного класса в метку класса
    predicted_class = index_to_label[predicted_class_index.item()]

    return predicted_class

In [ ]:
text = """"""

In [ ]:
classes = {
    'Договор': 10,
    'Заявление': 7,
    'Приказ': 4,
    'Соглашение': 6,
    'Устав': 0,
    'Доверенность': 8,
    'Акт': 1,
    'Решение': 9,
    'Оферта': 5,
    'Счет': 3,
    'bill': 2
}

model_path = 'trained_model_final.pth'

# Создание словаря для сопоставления меток классов с индексами
label_to_index = {label: index for label, index in classes.items()}

analyze_text(text, model_path, label_to_index)